In [1]:
%%capture
!pip install yfinance
!pip install "redshift-connector[full]" sqlalchemy-redshift
!pip install python-dotenv
!pip install psycopg2-binary

In [2]:
import os
from configparser import ConfigParser
import sqlalchemy as sa
from sqlalchemy.engine.url import URL
from sqlalchemy.exc import SQLAlchemyError, ResourceClosedError
import pandas as pd
from pathlib import Path
import yfinance as yf
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# This file is used by dotenv to load the db credentials
env_file = Path('.').resolve() / '.env'

In [6]:
# DB configuration environment variables
user = os.getenv('USERNAME')
passw = os.getenv('PASSW')
host = os.getenv('HOST')
port = os.getenv('PORT')
dbname = os.getenv('DB_NAME')
schema = "norbermv_dev_coderhouse"

In [7]:
### Defining some useful functions for handling the Redshift db connection.

In [8]:
def build_conn_string(
    user: str, 
    passw: str, 
    host: str, 
    port: str, 
    dbname: str
) -> URL:
    conn_string = URL.create(
        drivername="postgresql",
        username=user,
        password=passw,
        host=host,
        port=port,
        database=dbname
    )
    return conn_string

def conn_to_db(conn_str: URL) -> tuple :
    # Create an engine with the connection string
    engine = sa.create_engine(conn_str)
    try:
        # Connect to the database
        conn = engine.connect()
        return conn, engine
    except SQLAlchemyError as e:
        print(f"Error connecting to the database: {e}")
        return None, None

In [9]:
# Build the connection string, and connect to the DB
conn_str = build_conn_string(
    user, 
    passw, 
    host, 
    port,
    dbname
)

In [11]:
conn, engine = conn_to_db(conn_str)

## Create The `stock_hist`  Table

In [13]:
if conn is not None:
    try:
        # Use the connection to execute a DDL statement
        with conn.begin() as trans:
            conn.execute(
                f"""
                DROP TABLE IF EXISTS {schema}.stock_hist;
                CREATE TABLE {schema}.stock_hist (
                    Date TIMESTAMP,
                    open_price FLOAT,
                    High FLOAT,
                    Low FLOAT,
                    Close FLOAT,
                    Volume INT,
                    Dividends FLOAT,
                    Stock_Splits FLOAT
                );
                """
            )
            # Commit the transaction to ensure DDL statement is executed
            trans.commit()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()
else:
    print("Failed to connect to the database.")


/var/folders/67/9k7m5nln4qz0y9s4rgmd3s2c0000gn/T/ipykernel_9360/3873696306.py:5: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  conn.execute(


## Retrieving the data from the `yfinance` API
Using the yfinance API to fetch historical stock data for Apple Inc. (ticker symbol: AAPL)

In [14]:
# Create a Ticker object for Apple Inc.
aapl = yf.Ticker("AAPL")

In [16]:
# Get historical market data for Apple
aapl_hist = aapl.history(period="1y")

# Show meta information about the history for Apple (requires history() to be called first)
aapl_history_metadata = aapl.history_metadata

In [17]:
# Rename the aapl_hist DataFrame columns to match the column names in the 'stock_hist' table
aapl_hist.columns = ['open_price', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock_Splits']

In [18]:
aapl_hist.head()

,open_price,High,Low,Close,Volume,Dividends,Stock_Splits
Date,,,,,,,
2023-01-27 00:00:00-05:00,142.364244,146.411612,142.284686,145.118835,70555800,0.0,0.0
2023-01-30 00:00:00-05:00,144.154251,144.740968,142.055979,142.205139,64015300,0.0,0.0
2023-01-31 00:00:00-05:00,141.906802,143.537686,141.489139,143.487961,65874500,0.0,0.0
2023-02-01 00:00:00-05:00,143.169752,145.795077,140.534488,144.621628,77663600,0.0,0.0
2023-02-02 00:00:00-05:00,148.072348,150.339674,147.346410,149.981689,118339000,0.0,0.0


In [22]:
conn, engine = conn_to_db(conn_str)

In [23]:

if conn is not None:
    try:
        with conn.begin() as trans:
            for index, row in aapl_hist.iterrows():
                sql = f"""
                INSERT INTO {schema}.stock_hist (Date, open_price, High, Low, Close, Volume, Dividends, Stock_Splits)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
                """
                conn.execute(sql, (index, row['open_price'], row['High'], row['Low'], row['Close'], row['Volume'], row['Dividends'], row['Stock_Splits']))
            # Commit the transaction
            trans.commit()
    except Exception as e:
        if isinstance(e, ResourceClosedError):
            print("Failed to connect to the database");
        else:
            print(f"An error occurred: {e}")
    finally:
        conn.close()